<p style="text-align: center;"><font size="+2" color="navy"><b><u>ASIAN OPTION PRICING USING GEOMETRIC BROWNIAN MOTION</u></b></font></p>
<br>
<p style="text-align: center;"><b>VIPUL AMBASHT &emsp; &emsp;&emsp;&emsp;</b></p>

<b><u>Asian Option</u></b>

Asian option is an option where the payoff is determined based on the average price of the underlying asset over a certain period of time called Averaging period.

Asian options are less sensitive to short-term price fluctuations and can be a good way to hedge against volatility.

There are two types of Asian option - Fixed Strike and Floating Strike.

<b>Fixed Strike</b> 

The Strike price is fixed and the payoff is determined by the average price over the period.

Call Payout = maximum(0, A-E)

Put Payout = maximum(0, E-A)

E is the strike price and A is the average Price over the Averaging period.


<b>Floating Strike</b>

The strike price is floating and is equal to the average price over the period.

Call Payout = maximum(0, S[T] - A)

Put Payout = maximum(0, A - S[T])

E is the strike price and S[T] is the simulated price at t = T (maturity date).

<b><u>Geometric Brownian Motion</u></b>

Geometric Brownian Motion (GBM) is a mathematical model used to describe the random fluctuations of an asset's price over time.

GBM applies it to the logarithm of an asset's price. This means the price itself fluctuates exponentially, not linearly.

The Geometric Brownian Motion is of the form SDE -

$\frac{d S_t}{d S}=r d t+\sigma d W_t$

The solution of 3 forms for the above SDE are -

<p style="text-align: left;">1) Closed form or Exact solution:</p>

$S_T=S_0 \exp \left\{\left(r-\frac{1}{2} \sigma^2\right) T+\sigma \phi \sqrt{T}\right\}$<br>
for time horizon $T$ with $\phi \sim N(0,1)$<br>
and $W_t \sim N(0, t)$ can be written as $\phi \sqrt{T}$<br>

In time stepping form -<br>
$S_{t+\delta t}=S_t \exp \left\{\left(r-\frac{1}{2} \sigma^2\right) \delta t+\sigma \phi\sqrt{\delta t}\right\}$

<p style="text-align: left;">2) Euler-Maruyama (E-M) Scheme:</p>
    
In time stepping form - <br>
$
\begin{aligned}
& S_{t+\delta t}=S_t(1+r \delta t+\sigma \phi \sqrt{\delta t}) \\
\end{aligned}
$


<p style="text-align: left;">3) Milstein Scheme:</p>

It adds Milstein correction i.e. 
$
\frac{1}{2} \sigma^2\left(\phi^2-1\right) \delta t
$ to E-M

In time stepping form - <br>
$
\begin{aligned}
& S_{t+\delta t}=S_t(1+r \delta t+\sigma \phi \sqrt{\delta t} + \frac{1}{2} \sigma^2\left(\phi^2-1\right) \delta t)
\end{aligned}
$


Closed form solution assumes constant parameters and provides direct and exact solution. E-M and Milstein schemes are numerical methods and can simulate GBM for any set of values, but they introduce approximation errors that decrease with time step. Milstein scheme is more accurate with higher order of convergence than E-M, but it requires more computational resource than E-M scheme.


<u><b>APPROACH OF THE PROBLEM IN PYTHON CODE -</b></u>

In Python we will define simulate_path for the above 3 methods:

1) Exact solution or Closed Form Solution -> S[i+1] = S[i] * exp((r - 0.5 * sigma * sigma) * dt + sigma * sqrt(dt) * w)

2) E-M solution -> S[i+1] = S[i] * (1 + r * dt + sigma * sqrt(dt) * w)

3) Milstein solution -> S[i+1] = S[i] * (1 + r * dt + sigma * sqrt(dt) * w + 0.5 * sigma * sigma * (w * w - 1) * dt)

where w is the wiener process ~ N(0,1) (normally distributed with mean 0, and standard deviation 1)

<br/>

Then find the Asian Option payoff for each 1,2 and 3 above in following manner -

STEP 1 - Find Average value A(T).

There are 4 ways to find A(T):

i) Arithmetic average - Discrete Sampling -> A = $ \frac{1}{n} \sum_{i=1}^n S\left(t_i\right) $

ii) Arithmetic average - Continuous Sampling -> A = $ \frac{1}{T} \int_0^T S(t) d t $

iii) Geometric average - Discrete Sampling -> A = $ \left(\prod_{i=1}^n S\left(t_i\right)\right)^{1 / n} $

iv) Geometric average - Continuous Sampling -> A = $ \exp \left(\frac{1}{T} \int_0^T \ln (S(t)) d t\right) $

<br/>

STEP 2 - Find the Value V(T) of Call and Put at t=T and from it get value of V(0) at t= 0 as $ e^{-r T} $V(T).

There are two types of Asian option - Fixed Strike and Floating Strike. The value is found as below  -

For Fixed Strike -

C(0) = exp(-r * T) * mean(maximum(0, A-E))

P(0) = exp(-r * T) * mean(maximum(0, E-A))


For Floating Strike -

C(0) = exp(-r * T) * mean(maximum(0, S[T] - A))

P(0) = exp(-r * T) * mean(maximum(0, A - S[T]))

E is the strike price and S[T] is the simulated price at t = T.

<br/>

STEP 3 - Vary the value of E and T in the above Steps to see the effect -

E = 80, 90, 100, 110, 120

T = 0.25, 0.5, 1, 1.5, 2.0 yrs



In [1]:
# Importing libraries
import pandas as pd
from numpy import *
import matplotlib.pyplot as plt
import statistics
from scipy.stats import gmean

from math import cos, exp, pi, exp
from scipy.integrate import quad

<b>1) EXACT SOLUTION OR CLOSED FORM SOLUTION </b>

In [2]:
# define simulation function for exact solution

def simulate_path_exact(s0, mu, sigma, horizon, timesteps, n_sims):

    # set the random seed for reproducibility
    random.seed(10000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    
    S = zeros((t,n))
    S[0] = S0

    for i in range(0, t-1):
        w = random.standard_normal(n)
        S[i+1] = S[i] * (2.718281828459045 ** ((r - 0.5 * sigma * sigma) * dt + sigma * sqrt(dt) * w))

    return S
# using approx. vale of exp as (2.718281828459045 ^)

In [3]:
# Define parameters
E=100; T=1; r=0.05

# Call the simulation function - s0=100,  mu=rf=0.05(5%), sigma=vol=0.2(20%), horizon=T=T year, timesteps=252,
# n_sims=number of simultaions=100000
S = simulate_path_exact(100,0.05,0.2,T,252,100000)

print("Simulated Stock Price for 1 year period-")
S

Simulated Stock Price for 1 year period-


array([[100.        , 100.        , 100.        , ..., 100.        ,
        100.        , 100.        ],
       [ 98.42304727, 100.23409018,  99.63935908, ..., 100.00948691,
         99.53921833,  99.92087022],
       [ 97.06603649,  99.70458951,  97.63076597, ...,  99.0653992 ,
        101.47530605, 101.16121655],
       ...,
       [123.48139421,  75.68397358, 129.65785744, ..., 107.24642044,
        117.02921333,  88.09330239],
       [120.42557375,  74.54061262, 129.85757523, ..., 105.38005486,
        119.28547181,  90.15484782],
       [121.04176376,  74.27411087, 130.49423239, ..., 107.48385776,
        119.38576023,  86.89800955]])

Check Stock Price at T = 1 year -

In [4]:
S[251]

array([121.04176376,  74.27411087, 130.49423239, ..., 107.48385776,
       119.38576023,  86.89800955])

<b>i) Arithmetic average - Discrete Sampling</b>

In [5]:
def asian_arth_dis(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S = simulate_path_exact(100,0.05,0.2,T,timesteps,100000)
    # Average price
    A = S.mean(axis=0)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))

    return C0, P0, C1, P1


In [6]:
 # Print the values
C0, P0, C1, P1 = asian_arth_dis(100,1)
print(f"Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Floating Strike - Arithmetic average - Discrete Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Floating Strike - Arithmetic average - Discrete Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 5.7215
Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 3.3321
Asian Call Option Value for Floating Strike - Arithmetic average - Discrete Sampling is 5.8127
Asian Put Option Value for Floating Strike - Arithmetic average - Discrete Sampling is 3.4191


In [7]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [8]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_arth_dis(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)


In [9]:
print("Call Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,20.350260,20.720709,21.460755,22.224591,22.988561
90,10.522069,11.197748,12.551922,13.816962,14.983601
100,2.565844,3.816653,5.721539,7.288669,8.678395
110,0.155643,0.688925,1.968488,3.248592,4.476239
120,0.001808,0.065463,0.520574,1.241753,2.081514


In [10]:
print("Put Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.000014,0.002996,0.046697,0.131082,0.234792
90,0.047601,0.233133,0.650158,1.000888,1.278207
100,1.967154,2.605137,3.332069,3.750030,4.021375
110,9.432732,9.230509,9.091312,8.987388,8.867592
120,19.154674,18.360145,17.155692,16.257983,15.521242


In [11]:
print("Call Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,2.587944,3.870503,5.812748,7.536581,8.99344


In [12]:
print("Put Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,1.959015,2.619034,3.419134,3.846494,4.169033


<b>ii) Arithmetic average - Continuous Sampling</b>

For Integration of Stochastic equation we will use following concept -

Let us say F is a stochastic Function depending on dt and dw. The Integral with respect to dt will be the Expected value of Area under the curve F.

F being a stochastic process can follow different paths. So we will first find the Area for each path and then find mean of all the Areas belonging to different simulated paths (as given in image below).

Area under the curve can be calculated as sum of each strip (as given in image below).

![](Stochastic_Integration.png)

In [13]:
# define simulation function for exact solution and find Area A

def simulate_path_exact_integral_arth_cont(s0, mu, sigma, horizon, timesteps, n_sims):

    # set the random seed for reproducibility
    random.seed(10000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    
    S = zeros((t,n))
    A = zeros((t,n))
    S[0] = S0
    A[0] = 0

    for i in range(0, t-1):
        w = random.standard_normal(n)
        S[i+1] = S[i] * (2.718281828459045 ** ((r - 0.5 * sigma * sigma) * dt + sigma * sqrt(dt) * w))
        A[i] = S[i] * dt

    return S, A
# using approx. vale of exp as (2.718281828459045 ^)

Array of areas covered by different paths for T = 1 year -

In [14]:
# Define parameters
E=100; T=1; r=0.05

# Call the simulation function - s0=100,  mu=rf=0.05(5%), sigma=vol=0.2(20%), horizon=T=T year, timesteps=252,
# n_sims=number of simultaions=100000
S, S_area = simulate_path_exact_integral_arth_cont(100,0.05,0.2,T,252,100000)

S_area

array([[0.3968254 , 0.3968254 , 0.3968254 , ..., 0.3968254 , 0.3968254 ,
        0.3968254 ],
       [0.39056765, 0.39775433, 0.39539428, ..., 0.39686304, 0.3949969 ,
        0.39651139],
       [0.38518268, 0.39565313, 0.38742367, ..., 0.39311666, 0.40267979,
        0.4014334 ],
       ...,
       [0.49000553, 0.30033323, 0.51451531, ..., 0.42558103, 0.46440164,
        0.3495766 ],
       [0.47787926, 0.29579608, 0.51530784, ..., 0.41817482, 0.47335505,
        0.35775733],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

Check random Stock Prices created for the period T = 1 year -

In [15]:
S

array([[100.        , 100.        , 100.        , ..., 100.        ,
        100.        , 100.        ],
       [ 98.42304727, 100.23409018,  99.63935908, ..., 100.00948691,
         99.53921833,  99.92087022],
       [ 97.06603649,  99.70458951,  97.63076597, ...,  99.0653992 ,
        101.47530605, 101.16121655],
       ...,
       [123.48139421,  75.68397358, 129.65785744, ..., 107.24642044,
        117.02921333,  88.09330239],
       [120.42557375,  74.54061262, 129.85757523, ..., 105.38005486,
        119.28547181,  90.15484782],
       [121.04176376,  74.27411087, 130.49423239, ..., 107.48385776,
        119.38576023,  86.89800955]])

In [16]:
def asian_arth_cont(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S, S_area = simulate_path_exact_integral_arth_cont(100,0.05,0.2,T,timesteps,100000)
    # Average price
    A = (1/T)*S_area.sum(axis=0)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))

    return C0, P0, C1, P1

In [17]:
 # Print the values
C0, P0, C1, P1 = asian_arth_cont(100,1)
print(f"Asian Call Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Float Strike - Arithmetic average - Continuous Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Float Strike - Arithmetic average - Continuous Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is 5.4769
Asian Put Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is 3.4838
Asian Call Option Value for Float Strike - Arithmetic average - Continuous Sampling is 6.0575
Asian Put Option Value for Float Strike - Arithmetic average - Continuous Sampling is 3.2675


In [18]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [19]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_arth_cont(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)


In [20]:
print("Call Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,18.763226,19.928211,21.068498,21.965859,22.796531
90,8.981971,10.458329,12.199290,13.586340,14.812279
100,1.762933,3.364914,5.476864,7.117358,8.545145
110,0.074305,0.561033,1.849152,3.146309,4.387641
120,0.000625,0.048366,0.478770,1.192205,2.030571


In [21]:
print("Put Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.000050,0.004100,0.050892,0.136909,0.241106
90,0.094573,0.287317,0.693978,1.034824,1.305228
100,2.751313,2.947001,3.483846,3.843278,4.086468
110,10.938463,9.896219,9.368428,9.149663,8.977339
120,20.740561,19.136652,17.510341,16.472994,15.668643


In [22]:
print("Call Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,3.558469,4.34629,6.057525,7.706214,9.124067


In [23]:
print("Put Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,1.342471,2.301218,3.267458,3.751568,4.101316


<b>iii) Geometric average - Discrete Sampling</b>

In [24]:
def asian_geo_dis(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S = simulate_path_exact(100,0.05,0.2,T,timesteps,100000)
    # Geometric price
    A = gmean(S,axis=0)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))

    return C0, P0, C1, P1

In [25]:
 # Print the values
C0, P0, C1, P1 = asian_geo_dis(100,1)
print(f"Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Float Strike - Arithmetic average - Discrete Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Float Strike - Arithmetic average - Discrete Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 5.5061
Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 3.4506
Asian Call Option Value for Float Strike - Arithmetic average - Discrete Sampling is 6.0205
Asian Put Option Value for Float Strike - Arithmetic average - Discrete Sampling is 3.2929


In [26]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [27]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_geo_dis(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)

In [28]:
print("Call Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,20.267020,20.555182,21.138159,21.749595,22.362560
90,10.443983,11.051972,12.274184,13.402124,14.428385
100,2.518088,3.715751,5.506053,6.948182,8.206272
110,0.142792,0.639470,1.823779,2.992626,4.098389
120,0.001373,0.053859,0.447716,1.078860,1.810990


In [29]:
print("Put Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.000020,0.004124,0.058114,0.159970,0.282665
90,0.052760,0.254014,0.706433,1.089934,1.396864
100,2.002643,2.670891,3.450596,3.913428,4.223125
110,9.503125,9.347710,9.280617,9.235307,9.163616
120,19.237484,18.515197,17.416848,16.598975,15.924591


In [30]:
print("Call Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,2.634896,3.968925,6.020544,7.86465,9.445189


In [31]:
print("Put Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,1.922722,2.550801,3.292916,3.670678,3.946909


<b>iv) Geometric average - Continuous Sampling</b>

Using the previous Stochastic Integration method

In [32]:
# define simulation function for exact solution and find Area A
def simulate_path_exact_integral_geo_cont(s0, mu, sigma, horizon, timesteps, n_sims):

    # set the random seed for reproducibility
    random.seed(10000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    
    S = zeros((t,n))
    A = zeros((t,n))
    S[0] = S0
    A[0] = 0

    for i in range(0, t-1):
        w = random.standard_normal(n)
        S[i+1] = S[i] * (2.718281828459045 ** ((r - 0.5 * sigma * sigma) * dt + sigma * sqrt(dt) * w))
        A[i+1] = log(S[i]) * dt

    return S, A
# using approx. vale of exp as (2.718281828459045 ^)

Array of areas covered by different paths for T = 1 year -

In [33]:
# Define parameters
E=100; T=1; r=0.05

# Call the simulation function - s0=100,  mu=rf=0.05(5%), sigma=vol=0.2(20%), horizon=T=T year, timesteps=252,
# n_sims=number of simultaions=100000
S, S_area = simulate_path_exact_integral_geo_cont(100,0.05,0.2,T,252,100000)

S_area

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01827448, 0.01827448, 0.01827448, ..., 0.01827448, 0.01827448,
        0.01827448],
       [0.01821141, 0.01828376, 0.01826015, ..., 0.01827486, 0.01825616,
        0.01827134],
       ...,
       [0.01908771, 0.01706492, 0.01932477, ..., 0.01852167, 0.01887722,
        0.01776961],
       [0.01911147, 0.01716891, 0.01930516, ..., 0.0185521 , 0.01889851,
        0.01777141],
       [0.01901203, 0.01710851, 0.01931126, ..., 0.01848243, 0.01897428,
        0.01786321]])

Stock Price at T = 1 year -

In [34]:
S_area.sum(axis=0)

array([4.74502237, 4.50546438, 4.83370947, ..., 4.62435153, 4.8020367 ,
       4.50518359])

In [35]:
def asian_geo_cont(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S, S_area = simulate_path_exact_integral_geo_cont(100,0.05,0.2,T,timesteps,100000)
    # Average price
    A1 = (1/T)*S_area.sum(axis=0)
    
    A = power (2.718281828459045, A1)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))


    return C0, P0, C1, P1

In [36]:
 # Print the values
C0, P0, C1, P1 = asian_geo_cont(100,1)
print(f"Asian Call Option Value for Fixed Strike - Geometric average - Continuous Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Geometric average - Continuous Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Float Strike - Geometric average - Continuous Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Float Strike - Geometric average - Continuous Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Geometric average - Continuous Sampling is 4.4771
Asian Put Option Value for Fixed Strike - Geometric average - Continuous Sampling is 4.1999
Asian Call Option Value for Float Strike - Geometric average - Continuous Sampling is 7.1197
Asian Put Option Value for Float Strike - Geometric average - Continuous Sampling is 2.6138


In [37]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [38]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_geo_cont(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)

In [39]:
print("Call Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,13.255068,17.010002,19.386784,20.604470,21.519593
90,4.130194,7.858594,10.729878,12.396791,13.686684
100,0.284293,1.980210,4.477077,6.221562,7.641833
110,0.002227,0.230078,1.355180,2.576950,3.734955
120,0.000000,0.011146,0.299419,0.890510,1.613221


In [40]:
print("Put Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.003840,0.015489,0.085024,0.193760,0.317425
90,0.754744,0.617180,0.940413,1.263515,1.532890
100,6.784621,4.491895,4.199907,4.365722,4.536412
110,16.378333,12.494862,10.590303,9.998545,9.677909
120,26.251883,22.029030,19.046837,17.589539,16.604549


In [41]:
print("Call Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,7.94396,6.272656,7.11967,8.602147,10.002658


In [42]:
print("Put Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,0.216014,1.297987,2.613757,3.229261,3.626651


<b>2) E - M (EULER MARUYAMA) SOLUTION </b>

In [43]:
# define simulation function for exact solution
def simulate_path_em(s0, mu, sigma, horizon, timesteps, n_sims):

    # set the random seed for reproducibility
    random.seed(10000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    
    S = zeros((t,n))
    S[0] = S0

    for i in range(0, t-1):
        w = random.standard_normal(n)
        S[i+1] = S[i] * (1 + r * dt + sigma * sqrt(dt) * w)

    return S


In [44]:
# Define parameters
E=100; T=1; r=0.05

# Call the simulation function - s0=100,  mu=rf=0.05(5%), sigma=vol=0.2(20%), horizon=T=T year, timesteps=252,
# n_sims=number of simultaions=100000
S = simulate_path_em(100,0.05,0.2,T,252,100000)

print("Simulated Stock Price for Time T = 1 year -")
S

Simulated Stock Price for Time T = 1 year -


array([[100.        , 100.        , 100.        , ..., 100.        ,
        100.        , 100.        ],
       [ 98.41841759, 100.24175312,  99.64664371, ..., 100.01742297,
         99.54608997,  99.9287754 ],
       [ 97.05984028,  99.71876403,  97.62528921, ...,  99.07671355,
        101.47162225, 101.16951484],
       ...,
       [123.27112136,  75.68295491, 129.66007904, ..., 107.64255949,
        117.11539119,  88.06011089],
       [120.19190631,  74.53689167, 129.86993707, ..., 105.76135173,
        119.36111605,  90.10412529],
       [120.81487169,  74.27584132, 130.51540623, ..., 107.86036086,
        119.47089901,  86.79602161]])

Stock Price at Time T = 1 year -

In [45]:
S[251]

array([120.81487169,  74.27584132, 130.51540623, ..., 107.86036086,
       119.47089901,  86.79602161])

<b>i) Arithmetic average - Discrete Sampling</b>

In [46]:
def asian_arth_dis(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S = simulate_path_em(100,0.05,0.2,T,timesteps,100000)
    # Average price
    A = S.mean(axis=0)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))

    return C0, P0, C1, P1


In [47]:
 # Print the values
C0, P0, C1, P1 = asian_arth_dis(100,1)
print(f"Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Floating Strike - Arithmetic average - Discrete Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Floating Strike - Arithmetic average - Discrete Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 5.7214
Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 3.3323
Asian Call Option Value for Floating Strike - Arithmetic average - Discrete Sampling is 5.8127
Asian Put Option Value for Floating Strike - Arithmetic average - Discrete Sampling is 3.4192


In [48]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [49]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_arth_dis(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)


In [50]:
print("Call Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,20.350129,20.720503,21.460725,22.224727,22.988719
90,10.522499,11.198381,12.552682,13.817590,14.984130
100,2.565725,3.816489,5.721435,7.288427,8.678075
110,0.154405,0.687164,1.966595,3.246805,4.474580
120,0.001746,0.064753,0.518978,1.239869,2.079392


In [51]:
print("Put Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.000015,0.003075,0.047003,0.131648,0.235497
90,0.048163,0.234052,0.651255,1.001947,1.279282
100,1.967167,2.605259,3.332301,3.750218,4.021601
110,9.431625,9.229033,9.089756,8.986032,8.866480
120,19.154744,18.359721,17.154433,16.256530,15.519667


In [52]:
print("Call Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,2.587743,3.87037,5.812657,7.536395,8.992661


In [53]:
print("Put Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,1.959058,2.61904,3.419185,3.846688,4.168901


<b>ii) Arithmetic average - Continuous Sampling</b>

In [54]:
# define simulation function for exact solution and find Area A
def simulate_path_em_integral_arth_cont(s0, mu, sigma, horizon, timesteps, n_sims):

    # set the random seed for reproducibility
    random.seed(10000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    
    S = zeros((t,n))
    A = zeros((t,n))
    S[0] = S0
    A[0] = 0

    for i in range(0, t-1):
        w = random.standard_normal(n)
        S[i+1] = S[i] * (1 + r * dt + sigma * sqrt(dt) * w)
        A[i] = S[i] * dt

    return S, A
# using approx. vale of exp as (2.718281828459045 ^)

In [55]:
# Define parameters
E=100; T=1; r=0.05

# Call the simulation function - s0=100,  mu=rf=0.05(5%), sigma=vol=0.2(20%), horizon=T=T year, timesteps=252,
# n_sims=number of simultaions=100000
S, S_area = simulate_path_em_integral_arth_cont(100,0.05,0.2,T,252,100000)

S_area

array([[0.3968254 , 0.3968254 , 0.3968254 , ..., 0.3968254 , 0.3968254 ,
        0.3968254 ],
       [0.39054928, 0.39778473, 0.39542319, ..., 0.39689454, 0.39502417,
        0.39654276],
       [0.3851581 , 0.39570938, 0.38740194, ..., 0.39316156, 0.40266517,
        0.40146633],
       ...,
       [0.48917112, 0.30032919, 0.51452412, ..., 0.42715301, 0.46474362,
        0.34944488],
       [0.47695201, 0.29578132, 0.51535689, ..., 0.4196879 , 0.47365522,
        0.35755605],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [56]:
S

array([[100.        , 100.        , 100.        , ..., 100.        ,
        100.        , 100.        ],
       [ 98.41841759, 100.24175312,  99.64664371, ..., 100.01742297,
         99.54608997,  99.9287754 ],
       [ 97.05984028,  99.71876403,  97.62528921, ...,  99.07671355,
        101.47162225, 101.16951484],
       ...,
       [123.27112136,  75.68295491, 129.66007904, ..., 107.64255949,
        117.11539119,  88.06011089],
       [120.19190631,  74.53689167, 129.86993707, ..., 105.76135173,
        119.36111605,  90.10412529],
       [120.81487169,  74.27584132, 130.51540623, ..., 107.86036086,
        119.47089901,  86.79602161]])

In [57]:
def asian_arth_cont(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S, S_area = simulate_path_em_integral_arth_cont(100,0.05,0.2,T,timesteps,100000)
    # Average price
    A = (1/T)*S_area.sum(axis=0)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))

    return C0, P0, C1, P1

In [58]:
 # Print the values
C0, P0, C1, P1 = asian_arth_cont(100,1)
print(f"Asian Call Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Float Strike - Arithmetic average - Continuous Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Float Strike - Arithmetic average - Continuous Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is 5.4766
Asian Put Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is 3.4839
Asian Call Option Value for Float Strike - Arithmetic average - Continuous Sampling is 6.0576
Asian Put Option Value for Float Strike - Arithmetic average - Continuous Sampling is 3.2677


In [59]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [60]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_arth_cont(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)


In [61]:
print("Call Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,18.763105,19.928026,21.068472,21.966020,22.796685
90,8.982624,10.459042,12.200025,13.586996,14.812815
100,1.762178,3.364493,5.476627,7.117044,8.544816
110,0.073431,0.559360,1.847303,3.144484,4.385913
120,0.000599,0.047765,0.477239,1.190299,2.028475


In [62]:
print("Put Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.000055,0.004197,0.051200,0.137498,0.241804
90,0.095352,0.288313,0.695048,1.035908,1.306308
100,2.750684,2.946863,3.483944,3.843392,4.086683
110,10.937715,9.894829,9.366914,9.148266,8.976155
120,20.740661,19.136333,17.509144,16.471516,15.667091


In [63]:
print("Call Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,3.558918,4.346376,6.057582,7.706031,9.123327


In [64]:
print("Put Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,1.343169,2.301446,3.26766,3.751767,4.101226


<b>iii) Geometric average - Discrete Sampling</b>

In [65]:
def asian_geo_dis(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S = simulate_path_em(100,0.05,0.2,T,timesteps,100000)
    # Geometric price
    A = gmean(S,axis=0)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))

    return C0, P0, C1, P1

In [66]:
 # Print the values
C0, P0, C1, P1 = asian_geo_dis(100,1)
print(f"Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Float Strike - Arithmetic average - Discrete Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Float Strike - Arithmetic average - Discrete Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 5.5061
Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 3.4509
Asian Call Option Value for Float Strike - Arithmetic average - Discrete Sampling is 6.0203
Asian Put Option Value for Float Strike - Arithmetic average - Discrete Sampling is 3.2928


In [67]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [68]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_geo_dis(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)

In [69]:
print("Call Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,20.266910,20.555034,21.138235,21.749963,22.362981
90,10.444474,11.052719,12.275103,13.403001,14.429201
100,2.518046,3.715695,5.506119,6.948142,8.206209
110,0.141629,0.637827,1.822102,2.991063,4.096969
120,0.001319,0.053238,0.446297,1.077128,1.809202


In [70]:
print("Put Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.000021,0.004220,0.058464,0.160657,0.283468
90,0.053364,0.255003,0.707627,1.091129,1.398062
100,2.002714,2.671079,3.450937,3.913705,4.223444
110,9.502074,9.346310,9.279214,9.234061,9.162578
120,19.237543,18.514820,17.415704,16.597561,15.923185


In [71]:
print("Call Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,2.634613,3.968656,6.020269,7.864163,9.444132


In [72]:
print("Put Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,1.922702,2.550712,3.292846,3.670684,3.946664


<b>iv) Geometric average - Continuous Sampling</b>

Using the previous Stochastic Integration method

In [73]:
# define simulation function for exact solution and find Area A
def simulate_path_em_integral_geo_cont(s0, mu, sigma, horizon, timesteps, n_sims):

    # set the random seed for reproducibility
    random.seed(10000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    
    S = zeros((t,n))
    A = zeros((t,n))
    S[0] = S0
    A[0] = 0

    for i in range(0, t-1):
        w = random.standard_normal(n)
        S[i+1] = S[i] * (1 + r * dt + sigma * sqrt(dt) * w)
        A[i+1] = log(S[i]) * dt

    return S, A
# using approx. vale of exp as (2.718281828459045 ^)

In [74]:
# Define parameters
E=100; T=1; r=0.05

# Call the simulation function - s0=100,  mu=rf=0.05(5%), sigma=vol=0.2(20%), horizon=T=T year, timesteps=252,
# n_sims=number of simultaions=100000
S, S_area = simulate_path_em_integral_geo_cont(100,0.05,0.2,T,252,100000)

S_area

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01827448, 0.01827448, 0.01827448, ..., 0.01827448, 0.01827448,
        0.01827448],
       [0.01821122, 0.01828407, 0.01826044, ..., 0.01827518, 0.01825643,
        0.01827166],
       ...,
       [0.0190807 , 0.0170659 , 0.01932457, ..., 0.0185361 , 0.01887989,
        0.0177678 ],
       [0.01910471, 0.01716886, 0.01930522, ..., 0.01856673, 0.01890143,
        0.01776992],
       [0.01900432, 0.01710831, 0.01931164, ..., 0.01849677, 0.0189768 ,
        0.01786098]])

In [75]:
S_area.sum(axis=0)

array([4.74360612, 4.50520847, 4.83410562, ..., 4.62684645, 4.80215777,
       4.50535924])

In [76]:
def asian_geo_cont(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S, S_area = simulate_path_em_integral_geo_cont(100,0.05,0.2,T,timesteps,100000)
    # Average price
    A1 = (1/T)*S_area.sum(axis=0)
    
    A = power (2.718281828459045, A1)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))


    return C0, P0, C1, P1

In [77]:
 # Print the values
C0, P0, C1, P1 = asian_geo_cont(100,1)
print(f"Asian Call Option Value for Fixed Strike - Geometric average - Continuous Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Geometric average - Continuous Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Float Strike - Geometric average - Continuous Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Float Strike - Geometric average - Continuous Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Geometric average - Continuous Sampling is 4.4765
Asian Put Option Value for Fixed Strike - Geometric average - Continuous Sampling is 4.1996
Asian Call Option Value for Float Strike - Geometric average - Continuous Sampling is 7.1198
Asian Put Option Value for Float Strike - Geometric average - Continuous Sampling is 2.6141


In [78]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [79]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_geo_cont(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)

In [80]:
print("Call Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,13.255075,17.009997,19.387004,20.604876,21.520053
90,4.131111,7.859406,10.730701,12.397632,13.687443
100,0.282962,1.979162,4.476537,6.221361,7.641600
110,0.002156,0.228904,1.353477,2.575286,3.733429
120,0.000000,0.010928,0.298273,0.888865,1.611415


In [81]:
print("Put Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.003946,0.015716,0.085513,0.194477,0.318261
90,0.755760,0.618224,0.941503,1.264668,1.534026
100,6.783389,4.491079,4.199634,4.365833,4.536557
110,16.378360,12.493921,10.588868,9.997192,9.676759
120,26.251983,22.029043,19.045958,17.588206,16.603120


In [82]:
print("Call Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,7.944808,6.273332,7.119797,8.601731,10.0017


In [83]:
print("Put Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,0.217139,1.298854,2.614095,3.229343,3.62651


<b>3) MILSTEIN SOLUTION </b>

In [84]:
# define simulation function for exact solution
def simulate_path_mi(s0, mu, sigma, horizon, timesteps, n_sims):

    # set the random seed for reproducibility
    random.seed(10000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    
    S = zeros((t,n))
    S[0] = S0

    for i in range(0, t-1):
        w = random.standard_normal(n)
        S[i+1] = S[i] * (1 + r * dt + sigma * sqrt(dt) * w + 0.5 * sigma * sigma * (w * w - 1) * dt)

    return S


In [85]:
# Define parameters
E=100; T=1; r=0.05

# Call the simulation function - s0=100,  mu=rf=0.05(5%), sigma=vol=0.2(20%), horizon=T=T year, timesteps=252,
# n_sims=number of simultaions=100000
S = simulate_path_mi(100,0.05,0.2,T,252,100000)

print("Simulated Stock Price -")
S

Simulated Stock Price -


array([[100.        , 100.        , 100.        , ..., 100.        ,
        100.        , 100.        ],
       [ 98.42330387, 100.23406284,  99.63940359, ..., 100.00948649,
         99.53927566,  99.92088036],
       [ 97.06649667,  99.70462871,  97.63119139, ...,  99.0655266 ,
        101.47501776, 101.1610494 ],
       ...,
       [123.47779315,  75.68925564, 129.65472186, ..., 107.24487867,
        117.02458949,  88.09548282],
       [120.42275284,  74.54599692, 129.8544119 , ..., 105.37886111,
        119.28035715,  90.1566545 ],
       [121.03885343,  74.2795088 , 130.49097633, ..., 107.48225661,
        119.38063017,  86.90088812]])

In [86]:
S[251]

array([121.03885343,  74.2795088 , 130.49097633, ..., 107.48225661,
       119.38063017,  86.90088812])

<b>i) Arithmetic average - Discrete Sampling</b>

In [87]:
def asian_arth_dis(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S = simulate_path_mi(100,0.05,0.2,T,timesteps,100000)
    # Average price
    A = S.mean(axis=0)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))

    return C0, P0, C1, P1


In [88]:
 # Print the values
C0, P0, C1, P1 = asian_arth_dis(100,1)
print(f"Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Floating Strike - Arithmetic average - Discrete Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Floating Strike - Arithmetic average - Discrete Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 5.7205
Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 3.3313
Asian Call Option Value for Floating Strike - Arithmetic average - Discrete Sampling is 5.8117
Asian Put Option Value for Floating Strike - Arithmetic average - Discrete Sampling is 3.4183


In [89]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [90]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_arth_dis(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)


In [91]:
print("Call Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,20.350201,20.720581,21.460451,22.224086,22.987855
90,10.521952,11.197434,12.551277,13.816057,14.982476
100,2.565368,3.815961,5.720535,7.287419,8.676932
110,0.155496,0.688505,1.967645,3.247428,4.474806
120,0.001804,0.065374,0.520151,1.240976,2.080417


In [92]:
print("Put Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.000014,0.002988,0.046634,0.130938,0.234568
90,0.047543,0.232940,0.649753,1.000344,1.277563
100,1.966737,2.604567,3.331306,3.749141,4.020393
110,9.432644,9.230210,9.090711,8.986584,8.866641
120,19.154729,18.360177,17.155511,16.257567,15.520627


In [93]:
print("Call Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,2.587463,3.869797,5.811719,7.535265,8.991894


In [94]:
print("Put Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Arithmetic average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,1.958599,2.618458,3.418348,3.845573,4.168002


<b>ii) Arithmetic average - Continuous Sampling</b>

In [95]:
# define simulation function for exact solution and find Area A
def simulate_path_mi_integral_arth_cont(s0, mu, sigma, horizon, timesteps, n_sims):

    # set the random seed for reproducibility
    random.seed(10000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    
    S = zeros((t,n))
    A = zeros((t,n))
    S[0] = S0
    A[0] = 0

    for i in range(0, t-1):
        w = random.standard_normal(n)
        S[i+1] = S[i] * (1 + r * dt + sigma * sqrt(dt) * w + 0.5 * sigma * sigma * (w * w - 1) * dt)
        A[i] = S[i] * dt

    return S, A
# using approx. vale of exp as (2.718281828459045 ^)

In [96]:
# Define parameters
E=100; T=1; r=0.05

# Call the simulation function - s0=100,  mu=rf=0.05(5%), sigma=vol=0.2(20%), horizon=T=T year, timesteps=252,
# n_sims=number of simultaions=100000
S, S_area = simulate_path_mi_integral_arth_cont(100,0.05,0.2,T,252,100000)

S_area

array([[0.3968254 , 0.3968254 , 0.3968254 , ..., 0.3968254 , 0.3968254 ,
        0.3968254 ],
       [0.39056867, 0.39775422, 0.39539446, ..., 0.39686304, 0.39499713,
        0.39651143],
       [0.38518451, 0.39565329, 0.38742536, ..., 0.39311717, 0.40267864,
        0.40143274],
       ...,
       [0.48999124, 0.30035419, 0.51450286, ..., 0.42557492, 0.46438329,
        0.34958525],
       [0.47786807, 0.29581745, 0.51529529, ..., 0.41817008, 0.47333475,
        0.3577645 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [97]:
S

array([[100.        , 100.        , 100.        , ..., 100.        ,
        100.        , 100.        ],
       [ 98.42330387, 100.23406284,  99.63940359, ..., 100.00948649,
         99.53927566,  99.92088036],
       [ 97.06649667,  99.70462871,  97.63119139, ...,  99.0655266 ,
        101.47501776, 101.1610494 ],
       ...,
       [123.47779315,  75.68925564, 129.65472186, ..., 107.24487867,
        117.02458949,  88.09548282],
       [120.42275284,  74.54599692, 129.8544119 , ..., 105.37886111,
        119.28035715,  90.1566545 ],
       [121.03885343,  74.2795088 , 130.49097633, ..., 107.48225661,
        119.38063017,  86.90088812]])

In [98]:
def asian_arth_cont(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S, S_area = simulate_path_mi_integral_arth_cont(100,0.05,0.2,T,timesteps,100000)
    # Average price
    A = (1/T)*S_area.sum(axis=0)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))

    return C0, P0, C1, P1

In [99]:
 # Print the values
C0, P0, C1, P1 = asian_arth_cont(100,1)
print(f"Asian Call Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Float Strike - Arithmetic average - Continuous Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Float Strike - Arithmetic average - Continuous Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is 5.4759
Asian Put Option Value for Fixed Strike - Arithmetic average - Continuous Sampling is 3.4831
Asian Call Option Value for Float Strike - Arithmetic average - Continuous Sampling is 6.0565
Asian Put Option Value for Float Strike - Arithmetic average - Continuous Sampling is 3.2667


In [100]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [101]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_arth_cont(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)


In [102]:
print("Call Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,18.763169,19.928083,21.068192,21.965351,22.795822
90,8.981819,10.457989,12.198633,13.585429,14.811149
100,1.762476,3.364226,5.475862,7.116110,8.543683
110,0.074218,0.560661,1.848331,3.145158,4.386218
120,0.000623,0.048294,0.478369,1.191447,2.029490


In [103]:
print("Put Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.000050,0.004091,0.050825,0.136760,0.240877
90,0.094478,0.287096,0.693560,1.034272,1.304578
100,2.750913,2.946432,3.483083,3.842388,4.085486
110,10.938433,9.895966,9.367847,9.148871,8.976395
120,20.740616,19.136698,17.510179,16.472595,15.668041


In [104]:
print("Call Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,3.558006,4.345587,6.056497,7.704897,9.122521


In [105]:
print("Put Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Arithmetic average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,1.342074,2.300647,3.266674,3.750648,4.100288


<b>iii) Geometric average - Discrete Sampling</b>

In [106]:
def asian_geo_dis(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S = simulate_path_mi(100,0.05,0.2,T,timesteps,100000)
    # Geometric price
    A = gmean(S,axis=0)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))

    return C0, P0, C1, P1

In [107]:
 # Print the values
C0, P0, C1, P1 = asian_geo_dis(100,1)
print(f"Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Float Strike - Arithmetic average - Discrete Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Float Strike - Arithmetic average - Discrete Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 5.5051
Asian Put Option Value for Fixed Strike - Arithmetic average - Discrete Sampling is 3.4498
Asian Call Option Value for Float Strike - Arithmetic average - Discrete Sampling is 6.0194
Asian Put Option Value for Float Strike - Arithmetic average - Discrete Sampling is 3.2922


In [108]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [109]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_geo_dis(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)

In [110]:
print("Call Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,20.266994,20.555118,21.137974,21.749257,22.362075
90,10.443893,11.051707,12.273634,13.401363,14.427455
100,2.517631,3.715098,5.505130,6.947059,8.204984
110,0.142656,0.639080,1.823006,2.991574,4.097116
120,0.001370,0.053784,0.447346,1.078182,1.810038


In [111]:
print("Put Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.000020,0.004115,0.058039,0.159797,0.282399
90,0.052696,0.253803,0.705993,1.089337,1.396152
100,2.002212,2.670293,3.449784,3.912469,4.222056
110,9.503015,9.347375,9.279953,9.234418,9.162562
120,19.237507,18.515177,17.416588,16.598461,15.923858


In [112]:
print("Call Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,2.634397,3.968182,6.019439,7.863214,9.44348


In [113]:
print("Put Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Geometric average - Discrete Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,1.922321,2.550253,3.292185,3.669835,3.945978


<b>iv) Geometric average - Continuous Sampling</b>

Using the previous Stochastic Integration method

In [114]:
# define simulation function for exact solution and find Area A
def simulate_path_mi_integral_geo_cont(s0, mu, sigma, horizon, timesteps, n_sims):

    # set the random seed for reproducibility
    random.seed(10000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    
    S = zeros((t,n))
    A = zeros((t,n))
    S[0] = S0
    A[0] = 0

    for i in range(0, t-1):
        w = random.standard_normal(n)
        S[i+1] = S[i] * (1 + r * dt + sigma * sqrt(dt) * w)
        A[i+1] = log(S[i]) * dt

    return S, A
# using approx. vale of exp as (2.718281828459045 ^)

In [115]:
# Define parameters
E=100; T=1; r=0.05

# Call the simulation function - s0=100,  mu=rf=0.05(5%), sigma=vol=0.2(20%), horizon=T=T year, timesteps=252,
# n_sims=number of simultaions=100000
S, S_area = simulate_path_mi_integral_geo_cont(100,0.05,0.2,T,252,100000)

S_area

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01827448, 0.01827448, 0.01827448, ..., 0.01827448, 0.01827448,
        0.01827448],
       [0.01821122, 0.01828407, 0.01826044, ..., 0.01827518, 0.01825643,
        0.01827166],
       ...,
       [0.0190807 , 0.0170659 , 0.01932457, ..., 0.0185361 , 0.01887989,
        0.0177678 ],
       [0.01910471, 0.01716886, 0.01930522, ..., 0.01856673, 0.01890143,
        0.01776992],
       [0.01900432, 0.01710831, 0.01931164, ..., 0.01849677, 0.0189768 ,
        0.01786098]])

In [116]:
S_area.sum(axis=0)

array([4.74360612, 4.50520847, 4.83410562, ..., 4.62684645, 4.80215777,
       4.50535924])

In [117]:
def asian_geo_cont(E,T):
    
    r = 0.05
    timesteps = int (252 * T)
    S, S_area = simulate_path_mi_integral_geo_cont(100,0.05,0.2,T,timesteps,100000)
    # Average price
    A1 = (1/T)*S_area.sum(axis=0)
    
    A = power (2.718281828459045, A1)
    
    C0 = exp(-r*T) * mean(maximum(0, A-E))
    P0 = exp(-r*T) * mean(maximum(0, E-A))
    
    C1 = exp(-r*T) * mean(maximum(0, S[timesteps-1]-A))
    P1 = exp(-r*T) * mean(maximum(0, A-S[timesteps-1]))


    return C0, P0, C1, P1

In [118]:
 # Print the values
C0, P0, C1, P1 = asian_geo_cont(100,1)
print(f"Asian Call Option Value for Fixed Strike - Geometric average - Continuous Sampling is {C0:0.4f}")
print(f"Asian Put Option Value for Fixed Strike - Geometric average - Continuous Sampling is {P0:0.4f}")
print(f"Asian Call Option Value for Float Strike - Geometric average - Continuous Sampling is {C1:0.4f}")
print(f"Asian Put Option Value for Float Strike - Geometric average - Continuous Sampling is {P1:0.4f}")

Asian Call Option Value for Fixed Strike - Geometric average - Continuous Sampling is 4.4765
Asian Put Option Value for Fixed Strike - Geometric average - Continuous Sampling is 4.1996
Asian Call Option Value for Float Strike - Geometric average - Continuous Sampling is 7.1198
Asian Put Option Value for Float Strike - Geometric average - Continuous Sampling is 2.6141


In [119]:
# Vary the value of E and T in the above Steps to see the effect

E = [80,90,100,110,120]
T = [0.25,0.5,1,1.5,2]

In [120]:
c0_arr=[]
p0_arr=[]
c1_arr=[]
p1_arr=[]
data_call_fixed=[]
data_put_fixed=[]
data_call_float=[]
data_put_float=[]

for j in range(0, 5):
    for k in range(0, 5):
        c0, p0, c1, p1 = asian_geo_cont(E[j],T[k])
        c0_arr.append(c0)
        p0_arr.append(p0)
        c1_arr.append(c1)
        p1_arr.append(p1)
        x0=[E[j],T[k],c0]
        y0=[E[j],T[k],p0]
        x1=[T[k],c1]
        y1=[T[k],p1]
        data_call_fixed.append(x0)
        data_put_fixed.append(y0)
        data_call_float.append(x1)
        data_put_float.append(y1)

In [121]:
print("Call Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_fixed, columns=['Strike Price','Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', index='Strike Price', columns='Time to Maturity') 

Call Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,13.255075,17.009997,19.387004,20.604876,21.520053
90,4.131111,7.859406,10.730701,12.397632,13.687443
100,0.282962,1.979162,4.476537,6.221361,7.641600
110,0.002156,0.228904,1.353477,2.575286,3.733429
120,0.000000,0.010928,0.298273,0.888865,1.611415


In [122]:
print("Put Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_fixed, columns=['Strike Price','Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', index='Strike Price', columns='Time to Maturity') 

Put Price for Fixed Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Strike Price,,,,,
80,0.003946,0.015716,0.085513,0.194477,0.318261
90,0.755760,0.618224,0.941503,1.264668,1.534026
100,6.783389,4.491079,4.199634,4.365833,4.536557
110,16.378360,12.493921,10.588868,9.997192,9.676759
120,26.251983,22.029043,19.045958,17.588206,16.603120


In [123]:
print("Call Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_call = pd.DataFrame(data_call_float, columns=['Time to Maturity','Call Price'])

# create pivot table with x and rows, y and columns, z as values
df_call.pivot_table(values='Call Price', columns='Time to Maturity') 

Call Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Call Price,7.944808,6.273332,7.119797,8.601731,10.0017


In [124]:
print("Put Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -")       

# read it into a dataframe:
df_put = pd.DataFrame(data_put_float, columns=['Time to Maturity','Put Price'])

# create pivot table with x and rows, y and columns, z as values
df_put.pivot_table(values='Put Price', columns='Time to Maturity') 

Put Price for Float Strike - Geometric average - Continuous Sampling with varying Strike Price and Time to Maturity is -


Time to Maturity,0.25,0.50,1.00,1.50,2.00
Put Price,0.217139,1.298854,2.614095,3.229343,3.62651


Observation and Inference -
1. Call Option - Call Price decreases as Strike Price Increases. As time to maturity increases the Call price increases.
2. Put Option - Put Price increases as Strike Price Increases. As time to maturity increases the Put price decreases for Strike greater than 100 (S0-Initial Stock Price) and increases for Strike Price less than equal to S0.

As time increases the probability of Stock price to go above the current Price increases as the drift term is mainly positive.

<u><b>REFERENCES FOR ANSWERS</b></u>

For answering the questions following materials were referred -

1. CQF tutorials and workings shared by the CQF Instructors.
2. CQF books provided.
3. ChatGPT
4. Gemini (previously Bard)